In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import os

In [4]:
data_dir = 'C:\\Users\\Akshay\\Downloads\\Machine Learning Competition\\Dockship\\Garbage Classification AI Challenge'

In [5]:
test_path = data_dir+'\\TEST\\'
train_path = data_dir+'\\TRAIN1\\'

In [7]:
train = pd.read_csv(os.path.join(train_path, 'COMBINED_TRAIN.csv'))
test = pd.read_csv(os.path.join(test_path, 'test.csv'))
train.head()

,Image,Class
0,cardboard0.jpg,carboard
1,cardboard1.jpg,carboard
2,cardboard10.jpg,carboard
3,cardboard100.jpg,carboard
4,cardboard101.jpg,carboard


In [ ]:
l1 = []
l2 = []
for i in os.listdir(train_path):
    img = plt.imread(train_path+i)
    i1, i2, colors = img.shape
    l1.append(i1)
    l2.append(i2)
print(np.mean(l1))
print(np.mean(l2))

In [8]:
train.shape

(2001, 2)

In [9]:
test.head()

,Image
0,000.jpg
1,001.jpg
2,002.jpg
3,003.jpg
4,004.jpg


In [10]:
test.shape

(300, 1)

In [11]:
from tqdm import tqdm
from keras.preprocessing import image

train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('TRAIN1/'+train['Image'][i], target_size=(384,512,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

  0%|          | 0/2001 [00:00<?, ?it/s]C:\Users\Akshay\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 2001/2001 [00:04<00:00, 484.95it/s]


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Class'] = le.fit_transform(train['Class'])

In [13]:
from keras.utils import to_categorical

y = train['Class'].values
y = to_categorical(y)

In [14]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('TEST/'+test['Image'][i], target_size=(384,512,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)


  0%|          | 0/300 [00:00<?, ?it/s]C:\Users\Akshay\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 300/300 [00:00<00:00, 445.51it/s]


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [16]:
from keras.models import Sequential, load_model    
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator 


In [32]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(384,512,1)))

##model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(6, activation='softmax'))


In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 382, 510, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 170, 32)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 127, 170, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 690880)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               88432768  
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                

In [34]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [35]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
50/50 [==============================] - 104s 2s/step - loss: 7.3664 - acc: 0.2225 - val_loss: 1.7863 - val_acc: 0.2319
Epoch 2/5
50/50 [==============================] - 102s 2s/step - loss: 1.7815 - acc: 0.2394 - val_loss: 1.7787 - val_acc: 0.2319
Epoch 3/5
50/50 [==============================] - 105s 2s/step - loss: 1.7735 - acc: 0.2394 - val_loss: 1.7718 - val_acc: 0.2319
Epoch 4/5
50/50 [==============================] - 103s 2s/step - loss: 1.7663 - acc: 0.2394 - val_loss: 1.7659 - val_acc: 0.2319
Epoch 5/5
50/50 [==============================] - 104s 2s/step - loss: 1.7843 - acc: 0.2369 - val_loss: 1.7605 - val_acc: 0.2319


In [36]:
prediction = model.predict_classes(test)


In [38]:
test1 = pd.read_csv('test.csv')
test1.head()

,Image
0,000.jpg
1,001.jpg
2,002.jpg
3,003.jpg
4,004.jpg


In [39]:
test1.head()

,Image
0,000.jpg
1,001.jpg
2,002.jpg
3,003.jpg
4,004.jpg


In [40]:
solution = pd.DataFrame({'Filename': test1['Image'], 'Labels':le.inverse_transform(prediction)})
solution.head()

,Filename,Labels
0,000.jpg,paper
1,001.jpg,paper
2,002.jpg,paper
3,003.jpg,paper
4,004.jpg,paper


In [41]:
solution.to_csv('Solution.csv', index = False)